In [3]:
import re
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv1D, BatchNormalization, Activation
from keras.layers import Embedding, Input, Dense, Dropout, Lambda, MaxPooling1D
from keras.optimizers import SGD
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [4]:
#len(ALPHABET)=68
ALPHABET = 'abcdefghijklmnopqrstuvwxyz0123456789-,;.!?:’"/|_#$%ˆ&*˜‘+=<>()[]{} '
FEATURE_LEN = 140 #maxlen
path = '../data/'
file=pd.read_csv('training.csv',encoding ='latin-1',header=None)
file.info()
file.drop([1,2,3,4],axis=1,inplace=True)
file.columns='Sentiment','Text'
TRAIN_DATA_FILE=file[:1520000]
TRAIN_DATA_FILE = TRAIN_DATA_FILE.sample(frac=0.30, random_state=99)
TEST_DATA_FILE=file[1520000:]
target=TEST_DATA_FILE['Sentiment']
TEST_DATA_FILE.drop(['Sentiment'],axis=1,inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
0    1600000 non-null int64
1    1600000 non-null int64
2    1600000 non-null object
3    1600000 non-null object
4    1600000 non-null object
5    1600000 non-null object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [5]:
ls

datalab@  sample_data/  training.csv


In [6]:
TRAIN_DATA_FILE.info()
TEST_DATA_FILE.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 456000 entries, 1041223 to 44300
Data columns (total 2 columns):
Sentiment    456000 non-null int64
Text         456000 non-null object
dtypes: int64(1), object(1)
memory usage: 10.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 1520000 to 1599999
Data columns (total 1 columns):
Text    80000 non-null object
dtypes: object(1)
memory usage: 625.1+ KB


In [0]:
def get_char_dict():
    char_dict={}
    for i,c in enumerate(ALPHABET):
        char_dict[c]=i+1
    return char_dict

def char2vec(text, max_length=FEATURE_LEN):
    char_dict = get_char_dict()
    data=np.zeros(max_length)
    
    for i in range(0, len(text)):
        if i >= max_length:
            return data
        
        elif text[i] in char_dict:
            data[i] = char_dict[text[i]]
        
        else:
            data[i]=68
    return data
    

def conv_shape(conv):
    return conv.get_shape().as_list()[1:]

replace_ip=re.compile(r'([0-9]+)(?:\.[0-9]+){3}',)
def text_to_wordlist(text, remove_stopwords=True, stem_words=False):
    # Clean the text, with the option to remove stopwords and to stem words.
    
    # Convert words to lower case and split them
    text = text.lower().split()
    
    
    
    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
    
    text = " ".join(text)
    
    
    #Replace IP address
    text=replace_ip.sub('',text)
    
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)


In [0]:
train_df = TRAIN_DATA_FILE
test_df = TEST_DATA_FILE

list_sentences_train = train_df['Text'].values
y = train_df['Sentiment'].values
for i in range(len(y)):
    if(y[i] in [0,1]):
        y[i]=0
    elif(y[i] in [2,3]):
        y[i]=1
    else:
        y[i]=2
list_sentences_test = test_df['Text'].values
data=[]
for text in list_sentences_train:
    data.append(char2vec(text_to_wordlist(text)))
data=np.array(data)

test_data = []
for text in list_sentences_test:
    test_data.append(char2vec(text_to_wordlist(text)))
test_data=np.array(test_data)

In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /content/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
test_data.shape

(80000, 145)

In [0]:
def ConvolutionalBlock(input_shape, num_filters):
    model=Sequential()

    #1st conv layer
    model.add(Conv1D(filters = num_filters, kernel_size = 3, strides = 1, padding = "same", input_shape = input_shape))
    model.add(BatchNormalization())
    model.add(Activation("relu"))

    #2nd conv layer
    model.add(Conv1D(filters = num_filters, kernel_size = 3, strides = 1, padding = "same"))
    model.add(BatchNormalization())
    model.add(Activation("relu"))

    return model


In [0]:
def vdcnn_model(num_filters, num_classes, sequence_max_length, num_chars, embedding_size, top_k, learning_rate=0.001):
    
    inputs=Input(shape=(sequence_max_length, ), dtype='int32', name='input')
    
    embedded_seq = Embedding(num_chars, embedding_size, input_length=sequence_max_length)(inputs)
    embedded_seq = BatchNormalization()(embedded_seq)
    #1st Layer
    conv = Conv1D(filters=64, kernel_size=3, strides=2, padding="same")(embedded_seq)
    
    #ConvBlocks
    for i in range(len(num_filters)):
        conv = ConvolutionalBlock(conv_shape(conv), num_filters[i])(conv)
        conv = MaxPooling1D(pool_size=3, strides=2, padding="same")(conv)
        
    def _top_k(x):
        x = tf.transpose(x, [0, 2, 1])
        k_max = tf.nn.top_k(x, k=top_k)
        return tf.reshape(k_max[0], (-1, num_filters[-1] * top_k))
    
    k_max = Lambda(_top_k, output_shape=(num_filters[-1] * top_k,))(conv)
    
    #fully connected layers
    fc1=Dense(512, activation='relu', kernel_initializer='he_normal')(k_max)
    fc1=Dropout(0.3)(fc1)
    fc2=Dense(512, activation='relu', kernel_initializer='he_normal')(fc1)
    fc2=Dropout(0.3)(fc2)
    out=Dense(num_classes, activation='sigmoid')(fc2)
    
    
    model = Model(inputs=inputs, outputs=out)
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [14]:
num_filters = [64, 128, 256, 512]
model=vdcnn_model(num_filters=num_filters, num_classes=6,num_chars=69, sequence_max_length=FEATURE_LEN,embedding_size=16,top_k=3)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 145)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 145, 16)           1104      
_________________________________________________________________
batch_normalization_1 (Batch (None, 145, 16)           64        
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 73, 64)            3136      
_________________________________________________________________
sequential_1 (Sequential)    (None, 73, 64)            25216     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 37, 64)            0         
_________________________________________________________________
sequential_2 (Sequential)    (None, 37, 128)           75008     
__________

In [0]:
from keras.utils import to_categorical
y= to_categorical(y, num_classes=3)

In [0]:
from keras.utils import np_utils
def limit_mem():
    import keras.backend as K
    K.get_session().close()
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    K.set_session(K.tf.Session(config=cfg))
    print('gpu memory cleaned')
    
def preds(k):
    from datetime import datetime
    y_temp = np.zeros((len(test_data), 3))
    y_pred = np.zeros((len(test_data), 3))
    i=0;
    from sklearn.model_selection import KFold
    kf = KFold(n_splits=k, random_state=2)

    for train_index, test_index in kf.split(data):
        limit_mem()
        start=datetime.now()
        print('fold====================>>>>>>>>>>',i+1)
        X_train, X_test = data[train_index], data[test_index]
        y_train, y_test = y[train_index], y[test_index]
        #np_utils.to_categorical(y_train, nb_classes=6)

        model = None
        num_filters = [64, 128, 256, 512]
        model=vdcnn_model(num_filters=num_filters, num_classes=3,num_chars=69, sequence_max_length=FEATURE_LEN,embedding_size=16,top_k=3)

        early_stopping =EarlyStopping(monitor='val_loss', patience=5)
        bst_model_path = 'cv10_best_weights'+str(i+1) + '.h5'
        model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)

        hist = model.fit(X_train, y_train, \
                validation_data=(X_test, y_test), \
                epochs=200, batch_size=256,callbacks=[early_stopping, model_checkpoint])

        bst_val_score = min(hist.history['val_loss'])
        print('bst_val_score',bst_val_score)

        model.load_weights(bst_model_path)
        #model.fit(data, y,epochs=2, batch_size=256, shuffle=True,)
        
        y_temp = model.predict([test_data], batch_size=256, verbose=1)
        y_pred+=y_temp
        end=datetime.now()
        print(" ")
        print('time taken for this fold', end-start)
        i+=1
    y_test_pred=y_pred/k
    return y_test_pred

In [17]:
y_test=preds(10)

gpu memory cleaned
fold====================>>>>>>>>>> 1
Train on 410400 samples, validate on 45600 samples
Epoch 1/200
410400/410400 [==============================] - 196s 478us/step - loss: 0.4116 - acc: 0.7806 - val_loss: 0.4460 - val_acc: 0.7879
Epoch 2/200
410400/410400 [==============================] - 188s 458us/step - loss: 0.3488 - acc: 0.8243 - val_loss: 0.3436 - val_acc: 0.8257
Epoch 3/200
410400/410400 [==============================] - 187s 456us/step - loss: 0.3348 - acc: 0.8346 - val_loss: 0.3442 - val_acc: 0.8270
Epoch 4/200
410400/410400 [==============================] - 190s 463us/step - loss: 0.3265 - acc: 0.8405 - val_loss: 0.4777 - val_acc: 0.7592
Epoch 5/200
410400/410400 [==============================] - 186s 454us/step - loss: 0.3205 - acc: 0.8445 - val_loss: 0.3317 - val_acc: 0.8369
Epoch 6/200
410400/410400 [==============================] - 184s 448us/step - loss: 0.3157 - acc: 0.8477 - val_loss: 0.3644 - val_acc: 0.8281
Epoch 7/200
410400/410400 [========